In [3]:
!curl -o /tmp/ozone-filesystem-hadoop3-2.1.0.jar https://repo1.maven.org/maven2/org/apache/ozone/ozone-filesystem-hadoop3/2.1.0/ozone-filesystem-hadoop3-2.1.0.jar
!curl -o /tmp/hadoop-aws-3.3.4.jar https://repo1.maven.org/maven2/org/apache/hadoop/hadoop-aws/3.3.4/hadoop-aws-3.3.4.jar
!curl -o /tmp/aws-java-sdk-bundle-1.12.262.jar https://repo1.maven.org/maven2/com/amazonaws/aws-java-sdk-bundle/1.12.262/aws-java-sdk-bundle-1.12.262.jar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 69.0M  100 69.0M    0     0  10.6M      0  0:00:06  0:00:06 --:--:-- 11.2M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  940k  100  940k    0     0  4065k      0 --:--:-- --:--:-- --:--:-- 4069k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  267M  100  267M    0     0  10.9M      0  0:00:24  0:00:24 --:--:-- 10.6M


In [1]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession \
    .builder \
    .appName("spark-test") \
    .config("spark.master", "local") \
    .config('spark.driver.cores', '1')\
    .config('spark.driver.memory', '1G')\
    .config('spark.executor.instances', '1')\
    .config('spark.executor.cores', '1')\
    .config('spark.executor.memory', '512m')\
    .config("spark.ui.port", "4045")\
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.jars", "/tmp/hadoop-aws-3.3.4.jar,/tmp/aws-java-sdk-bundle-1.12.262.jar") \
    .config("spark.hadoop.fs.defaultFS", "s3a://ozone-om") \
    .config("spark.hadoop.fs.s3a.access.key", "ozone-access-key") \
    .config("spark.hadoop.fs.s3a.secret.key", "ozone-secret-key") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://ozone-s3g:9878") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .enableHiveSupport() \
    .getOrCreate()
    #.config("spark.sql.warehouse.dir", "ofs://ozone-om/s3v/mybucket/warehouse/") \
    #.config("spark.sql.catalogImplementation", "hive")\
    #.config("spark.hadoop.hive.metastore.uris", "thrift://hive-metastore:9083")\

26/01/25 19:31:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [7]:
spark.sparkContext.uiWebUrl

'http://bfdf51c5de1a:4045'

In [ ]:
spark.sql("SHOW DATABASES;").show()

In [4]:
df = spark.createDataFrame([(1, "tgete", "2025-02-01"), (3, "vdsv", "2025-06-01")], ["id", "descr", "report_dt"])

In [5]:
df.show()

+---+-----+----------+
| id|descr| report_dt|
+---+-----+----------+
|  1|tgete|2025-02-01|
|  3| vdsv|2025-06-01|
+---+-----+----------+



In [6]:
df.write\
    .format("parquet") \
    .save("s3a://ozone-om/s3v/mybucket/test_db/test_table")

26/01/25 19:31:54 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


Py4JJavaError: An error occurred while calling o74.save.
: org.apache.hadoop.fs.s3a.UnknownStoreException: `s3a://ozone-om/s3v/mybucket/test_db/test_table': getFileStatus on s3a://ozone-om/s3v/mybucket/test_db/test_table: com.amazonaws.services.s3.model.AmazonS3Exception: The specified bucket does not exist (Service: Amazon S3; Status Code: 404; Error Code: NoSuchBucket; Request ID: bec54048-e4d3-4c99-a765-0b63c87d354e; S3 Extended Request ID: null; Proxy: null), S3 Extended Request ID: null:NoSuchBucket: The specified bucket does not exist (Service: Amazon S3; Status Code: 404; Error Code: NoSuchBucket; Request ID: bec54048-e4d3-4c99-a765-0b63c87d354e; S3 Extended Request ID: null; Proxy: null)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:263)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:175)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:3858)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStat

In [7]:
spark.stop()